In [ ]:
%pip install requests tqdm

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import collections
import shutil
import time
import requests
from concurrent import futures
from http import HTTPStatus

SERVER_URL = 'http://localhost:8001'

URL_FMT = SERVER_URL + '/flags/{cc}/{cc}.gif'

DEST_DIR = 'downloads/'

if os.path.exists(DEST_DIR):
    shutil.rmtree(DEST_DIR)
    os.mkdir(DEST_DIR)
else:
    os.mkdir(DEST_DIR)

ALL_CC = (
    'AD AM AZ BF BN BW CF CM CV DK EG FJ GD GQ HN IE IS KE KN LA LR LY '
    'MG MN MW NE NP PE PT RS SC SK SR SZ TL TT UG VC YE AE AO BA BG BO '
    'BY CG CN CY DM ER FM GE GR HR IL IT KG KP LB LS MA MH MR MX NG NR '
    'PG PW RU SD SL SS TD TM TV US VE ZA AF AR BB BH BR BZ CH CO CZ DZ '
    'ES FR GH GT HT IN JM KH KR LC LT MC MK MT MY NI NZ PH PY RW SE SM '
    'ST TG TN TW UY VN ZM AG AT BD BI BS CA CI CR DE EC ET GA GM GW HU '
    'IQ JO KI KW LI LU MD ML MU MZ NL OM PK QA SA SG SN SV TH TO TZ UZ '
    'VU ZW AL AU BE BJ BT CD CL CU DJ EE FI GB GN GY ID IR JP KM KZ LK '
    'LV ME MM MV NA NO PA PL RO SB SI SO SY TJ TR UA VA WS'.split()
)

Result = collections.namedtuple('Result', 'status cc')


def get_flag(cc):
    url = URL_FMT.format(cc=cc)
    resp = requests.get(url)
    if resp.status_code != 200:
        resp.raise_for_status()

    return resp.content


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def download_one(cc, verbose=False):
    try:
        image = get_flag(cc)
    except requests.exceptions.HTTPError as exc:
        res = exc.response
        if res.status_code == 404:
            status = HTTPStatus.NOT_FOUND
            msg = 'not found'
        else:
            raise
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.OK
        msg = 'OK'

    if verbose:
        print(cc, msg)

    return Result(status, cc)


def download_many(cc_list, verbose, concur_req):
    counter = collections.Counter()
    with futures.ThreadPoolExecutor(max_workers=concur_req) as executor:
        todo_map = {}
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc, verbose)
            todo_map[future] = cc

        done_iter = futures.as_completed(todo_map)
        for future in done_iter:
            try:
                res = future.result()
            except requests.exceptions.HTTPError as exc:
                error_msg = 'HTTP {res.status_code} - {res.reason}'
                error_msg = error_msg.format(res=exc.response)
            except requests.exceptions.ConnectionError as exc:
                error_msg = 'connection error'
            else:
                error_msg = ''
                status = res.status

            if error_msg:
                status = HTTPStatus.error

            counter[status] += 1
            if verbose and error_msg:
                cc = todo_map[future]
                print(f'*** error for {cc}: {error_msg}')

    return counter


def main(download_many, concur_req, max_concur_req):
    t0 = time.perf_counter()
    count = download_many(
        ALL_CC, True, min(concur_req, max_concur_req))
    elapsed = time.perf_counter() - t0
    print()
    print(f'donwloaded {count} flags in {elapsed:.4f}s')


main(download_many, 16, 16)

AU OK
AFAZ OK
AD  AEAM OKOK
 OK

OK
BE OK
AL OKBB OK

BABD  OK
OK
AG OK
BG OK
BF OK
AO OK
BIBH OK
AR OKBN OK
 BROK
BS OK

BO  AT OK
OK
BJ OK
OK
BY OKCABT OK
CF
BZ  OK
BW CIOK OKCG 

OKOK

 CROK
CD OK
CM OK
CH OK
 OKCOCNCU
 OK
  DKOK
 DE OK
CL OK
OKCZ OKCY
OK
 OK

EGDMCV OK
EC OK
DZEE OK
  OKDJ OKOK
 

OK
ESER OK
 OK
GAET OK
 OK
FIFR OK
 OKFJ OK

GN GE OK
GDOK
GQ OK
FMGM OK
  GBOK
GT OK
GR OK
GH OK
OK
HN OK
GYGW OK
HT OK
HU ID OK
OK  OK
OK

HR OK
IEILIN OK
IR OK
IQ OK
IT OK
 JPIS  OK
JMOKOK
 OK
KG
KEJO OK
 OK
KI OK
  OK
OKKN
 KM OK
KHLC OK
 OK
KW OKOK

LRKRLB OKLAKZ OK
 OK
 LIOK OK
LK 
OK OK


LTKP OK
 OK
LV OK
MA OK
ML OKMD OK

MC OK
MGLS OK
LYLU  OK OK
OK

MEMRMH OK
MT  OK MMOK
OK 

OK
MN OK
MX OKMU OK
MY MK OK
MVOK
 
MWOKNG
 OK
 OKNI OK
NENA OK
NLMZ NZ
NP OK
OK
   OK
NOOK
 OK
OKPE
OM PA OKOK
 OKNR
 OK

QA OK
PG OK
PY OK
PT OK
PHPW OKRW OK
PL OK
RO OK
 
PKOK RUSERS OK
OK
 OK
SCSA OK

 OK
SB SD OK
 OK
SG OK
OK
SO OK
SNSMSISL OK
   SKOK
OK
OK SS OK

OK
SRSY OK
SV OK
 OK
ST OK
SZTD OK
 O